# Theorical overview


## Reinforcement learning in a nutshell

From Stuart Russell & Peter Norvig [RusNor2010] :
> [Reinforcement learning is a branch of Artificial Intelligence] in which we examine how an agent can learn from success and failure, from reward and punishment.

From Wikipedia :
> Reinforcement learning is an area of machine learning [...] concerned with how software agents ought to *take actions* in an *environment* so as to *maximize some notion of cumulative reward*

The aim of RL is thus to develop optimal control agents which would learn to behave optimally by learning from the reward signal obtained when they interact with the environment.


### The Agent and the Environment

In typical RL applications, the Agent receive *perceptions* from the environment. Those perceptions are pieces of information from the environment internal state. The agent can also *act* on the environment. When it does so, it gets a *reward signal*  for the action taken. Acting on the environment can make its internal state change.

The typical Agent-Environment interaction pattern is shown in the picture below.

![he Agent-Environment interacting loop](pics/AgentEnvironment.png)

### Maximizing the cumulative reward

The goal of the learner is to act in a way that maximize the expected cumulative reward, *i.e* to maximize the total reward accross the lifetime of the agent which is defined as :

$$\mathbb{E}\left(\sum\limits_{t = 0}^{+\infty}\gamma^tr_t\right)$$

where $r_t$ is the reward that the agent received at iteration $t$ and $\gamma$ a discount factor which allows to tune the balance between long term and short term gain. It is a value between 0 and 1. When $\gamma$ is close to one, the actions and possibility far in the future ar taken into account. The extreme case, which is having a $\gamma$ value of 1, comes down to maximizing over the whole lifetime of the agent. At the opposite, a $\gamma$ value close to 0 doesn't consider long term consequencies of the action. The extreme case, a $\gamma$ value of 0, comes down to acting greedily with respect to the immediate reward, *i.e* choosing the action which brings the highest immediate reward with no consideration for the future.

In practice, even if we are interested with long-term optimal behaviour, the algorithms may require a $\gamma$ value slightly lower than 1 to converge.


Once the optimisation criterion fixed, there is still many methods available to actualy optimize it. 

## Artificial Neural Networks

As all the technics we used in our investigations were either based or at least using artificial neural networks, we present them quickly for the readers who might not know what it is about. Be aware that this is in no way a thorough introduction, it makes some approximation and doesn't cover the wide variety of artifical neurons. 

### Artificial neurons

The standard artificial neurons or units can be seen as mathematical function. A unit is defined by its number of inputs, its *activation function* and its *weights*. If the unit has $n_{\mathrm{inputs}}$ inputs and the value of the input $i$ is called $x_i$, then it has one weight per input (so the unit as $n_{\mathrm{inputs}} + 1$ weights as a special input with constant value of 1 which is called the bias is allways present) and the function computed by the unit is the composition of the activation function with a linear combination of the inputs with the unit weights. The expression of the unit output if the activation function of the unit is $f$ is thus

$$f\left(\sum\limits_{i\;=\;0}^{n_{\mathrm{inputs}}}w_i\times x_i\right)$$

### Artificial neural networks
An artificial neural networks is a set of units where the outputs of some units are used as the input of some others. A lot of architectures are available. The units which take for input real data are called input units, the ones from which we take the result are called output units and the other which live inside the network and are not directly connected to the exterior are called hidden unit.

It is worth to mention that in some architectures like convolutional neural networks, some unit share their weight in order to reduce the number of parameters of the model.

The NN can be seen as series of non-linear transformations of linear combinations of the inputs in each layer into which a bias is added.

![A fully connected artificial neural network with four inputs, one output and five hidden](pics/fig_neural_network_1.png) 

# Reinforcement learning for video games

## Introduction

The videogame field offers interesting properties for reinforcement learning algorithm testing. 
First of all the reward and actions are quite straightforward to determine. Secondly installation is much simpler than setting up for example a robotic test bench. And lastly, the simulation can be launched as many times as wanted.


### The Arcade Learning Environment

For our simulation we used the Arcade Learning Environment framework. This framework is widely used in the machine learning community. It offers the possibility to train our agent on half a hundred of atari classic games such as Pac-Man and breakout.

The Arcade Learning Environment allows to retrieve the screen pixel values and the score of the game. It does no processing so the inputs are raw inputs and the relevant features have to be extracted by the learner.

## The genetic approach

Our first approach used genetic algorithms to learn a game strategy. Genetic algorithms are a way of exploring a solution space inspired by the evolution theory. The best way to understand the underlying idea is to see them in action on a toy problem. 

Let's imagine a game that consist in guessing a 6 character word. A solution is rated by the number of letter that are correct.

The genetic algorithm will proceed by generating a batch of random candidates. Each candidate is then rated and the candidates of the next generation are produced by combining parts of the candidate of the previous generation. The probability for a given of beeing choosen for contributing to the configuration of a new candidate is proportional to its score. Other bio inspired features such as random mutations are implemented in genetic algorithms. Generation after generation, the score is getting better until a matching solution is found.

The following python program implements an genetic solution search on the toy example.

In [8]:
import random
import string
import timeit

#Change here the secret word. Please take care to choose an uppercase unaccentuated word !
secret       = "REIN"
solutionSize = len(secret)

nbCandidates = 200
subPopSize   = 40
probMutation = 0.05

def geneticSearch(printOutput):
        candidates = generateCandidates(nbCandidates)
        candidates.sort(key = rateCandidate, reverse = True)
        time_begin = timeit.default_timer()
        nb_iter = 0
        best_score   = rateCandidate(candidates[0])
        while best_score < solutionSize:
            newGeneration = []
            candidates = candidates[:subPopSize]
            for _ in range(nbCandidates):
                p1, p2 = random.sample(candidates, 2)
                newGeneration.append(crossoverMutate(p1,p2))
            newGeneration.sort(key=rateCandidate, reverse=True)
            candidates = newGeneration
            #print("Iteration ",nb_iter, " : best word : ",newGeneration[0])
            nb_iter += 1
            best_score = rateCandidate(newGeneration[0])
        time_end = timeit.default_timer()
        if printOutput:
            print('The secret word is', candidates[0], 'and was found in', nb_iter, 'iterations')
            print('solution found in', str(time_end - time_begin), 's')
        return nb_iter
        
    
def crossoverMutate(p1, p2):
    child = []
    rp1 = rateCandidate(p1) + 1
    rp2 = rateCandidate(p2) + 1
    mixRatio = rp1 / (rp1 + rp2)
    for c1, c2 in zip(p1,p2):
        if random.random() <= probMutation:
            child.append(random.choice(string.ascii_uppercase))
        else:
            if random.random() <= mixRatio:
                child.append(c1)
            else:
                child.append(c2)
    return ''.join(child)
    
def rateCandidate(candidate):
    score = 0
    for c1, c2 in zip(candidate, secret):
        if c1 == c2:
            score += 1
    return score

def generateCandidates(nbCandidates):
    candidates = []
    for _ in range(0, nbCandidates):
        randomString = ''.join(random.choice(string.ascii_uppercase) for _ in range(solutionSize))
        candidates.append(randomString)
    return candidates

def printCandidates(candidates):
    print(' '.join(cand for cand in candidates))

geneticSearch(True)

nbIters = [geneticSearch(False) for _ in range(25)]

print('Average number of iteration needed : ', sum(nbIters)/len(nbIters))


The secret word is REIN and was found in 2 iterations
solution found in 0.008064467000167497 s
Average number of iteration needed :  2.56


In order to see the performance of the algorithm, let's compare it with a random search with the same number of candidates per generation :

In [9]:
nb_max_iter = 10000
def randomSearch(printOutput):
    time_begin = timeit.default_timer()
    nb_iter    = 0
    best_score = 0
    while best_score < solutionSize and nb_iter < nb_max_iter:
        candidates   = generateCandidates(nbCandidates)
        maxCandidate = max(candidates, key=rateCandidate)
        best_score   = rateCandidate(maxCandidate)
        nb_iter += 1
    
    time_end = timeit.default_timer()
    if printOutput:
        if rateCandidate(maxCandidate) == solutionSize:
            print('The secret word is', maxCandidate, 'and was found in', nb_iter, 'iterations')
            print('solution found in', str(time_end - time_begin), 's')
        else:
            print('The solution was not found in', nb_max_iter, 'iterations')
    
    return nb_iter

randomSearch(True)
nbIters = [randomSearch(False) for _ in range(10)]

print('Average number of iteration needed : ', sum(nbIters)/len(nbIters))

The secret word is REIN and was found in 5171 iterations
solution found in 9.91076282899985 s
Average number of iteration needed :  2152.7


## Genetic evolution of neural networks topologies

The NEAT algorithm, introduced in 2002 by K. Stanley and R. Miikkulainen [StaMii2002] is an attempt to use genetic algorithm to determine adequate topology (number of nodes, number of layers, connections between layers) and weights to solve a problem using a neural network.

In this approach, the evolving network can be viewed as the internal circuitry of the controler. We do not try to understand it, it is just a way to control the game. 

To find the good configuration, the NEAT library starts with generating a lot of simple topologies. Those simple topologies are rated on their average final score on the game. Generation after generation, the best topologies are
mixed together to create new and more efficient topologies.

### Application of NEAT on enginered features

As the space search is huge and that those parameters are correlated, the NEAT algorithm isn't efficient on problems with a huge set of inputs. As we however wanted to try this algorithm, we develop our own version of the breakout game in which we had access to a smallest set of intelligent features such as the pad position and the ball position, the brick presence on the screen etc.

Une vidéo de l'agent obtenu est visible dans 

### Experiments

We try different sets of features and we study the influence of allowing the neural net to have recurrent connection. It appears that a small amount of reccurence tends to diminue the number of generation needed to find a solution.

We can note that if we allow the agent to take a lot of time to solve the game, the input set needed to get a solution can be very small and with simple inputs(normalised coordinates of the ball relative to the pad) a solution with as few as 27 unit and 50 links can be found. But as soon as we put a constraint on the time allowed to solve the game, the system complexity grows and a bigger input sets gives better results. The smallest solution which achieve to solve the game in a reasonable time (around human performance) used however around 300 units and 3000 links which is not very efficient for a such little number of inputs.

### Conclusions on NEAT

NEAT is an interesting approach but it is difficult to use it in a real world problem as it's solution complexity grow rapidly.

Research is currently be done to overcome this dificulty and evolutions of NEAT such as HyperNEAT are worth to investigate on.

## The deep $Q$-Learning approach

### Introduction to $Q$-learning

The $Q$-learning method is a reinforcement learning technic based on the so called $Q$ values. The $Q$-value of a state-action pair $(s,a)$ is the maximal expected cumulated (and possibly discounted) reward that an agent can get when it is in state $s$ and takes the action $a$.

The idea is to create an estimator $\hat{Q}$ of the $Q$-values of the different state-action pairs. When we have such an estimator, the optimal behaviour is to take at each step the action which has the maximal estimated $Q$-value for the current state $s$.

However, it is also important to explore the state-action space to improve the quality of the estimator.

The simplest way to achieve that is to follow an $\epsilon$-greedy policy which means that the action is choosen randomly for a proportion $\epsilon$ of the timesteps and with the above behaviour the rest of the time.

A $Q$-value table can be constructed for small action space problems and filled during the exploration.

Let's implement it for the problem of finding the nearest power point on a corridor. 

The corridor is modeled as an array of 20 cells. There are power point at each end of the corridor.

The agent get a reward of -1 for each step and the cells with power point on them are terminal states ($i.e$ which ends the exploration).

In [4]:
import numpy as np
from random import random, choice

corridorLength = 20
initExplor     = 10
epsilon        =  0.05
gamma          =  0.95
nbAgents       = 201

class QTable:
    def __init__(self):
        self.values = np.random.ranf([corridorLength, 2])
        self.values[[0, corridorLength-1]] = 0
    
    def getAction(self, x):
        return np.argmax(self.values[x])
    
    def getValue(self, x):
        return np.max(self.values[x])
    
    def __str__(self):
        chars = ['S']
        for i in range(1,corridorLength-1):
            if self.values[i][0] > self.values[i][1]:
                chars.append('←')
            else:
                chars.append('→')
        chars.append('S')
        return " ".join(chars)
    
    def getVValues(self):
        maximas = np.maximum.reduce(self.values, axis=1)
        return " ".join("{0:.2f}".format(val) for val in maximas)

def explore(qtable):
    newX = choice(range(1, corridorLength - 1))
    while newX == 10:
        newX = choice(range(1, corridorLength - 1))
    x = newX
    while x != 0 and x != corridorLength-1 :
        rnd = random()
        if rnd < epsilon:
            if random() < 0.5:
                action = 0
            else:
                action = 1
        else:
            action = qtable.getAction(x)

        newX = x + 2*action - 1
        #update table
        qtable.values[x][action] = -1 + gamma * qtable.getValue(newX)
        x = newX
                
    
qtable = QTable()
print("Init", qtable)
i = 0
nb_void_iter = 0
old_qtable_val = np.copy(qtable.values)

while nb_void_iter < 30:
    np.copyto(old_qtable_val, qtable.values)
    
    explore(qtable)
    np.subtract(old_qtable_val, qtable.values, old_qtable_val)
    if np.max(np.abs(old_qtable_val)) < 0.0001:
        nb_void_iter += 1
    else:
        nb_void_iter = 0
    if i%5 == 0:
        print("Iteration {0:>4d} : ".format(i), qtable)
    i+= 1

print('End',qtable)
print('Estimated V-Value : \n', qtable.getVValues())

Init S → → ← ← → ← → ← ← ← ← → ← ← → ← → ← S
Iteration    0 :  S ← ← ← ← → ← ← ← → ← ← → → → → ← → ← S
Iteration    5 :  S ← ← → ← ← → → → → ← ← → → → → → → → S
Iteration   10 :  S ← ← → ← ← → → → → ← ← → → → → → → → S
Iteration   15 :  S ← ← ← ← ← → → ← → → → → → ← → → → → S
Iteration   20 :  S ← ← ← ← ← → → ← ← ← ← → → → → → → → S
Iteration   25 :  S ← ← ← ← ← ← ← ← ← ← → → → → → → → → S
Iteration   30 :  S ← ← ← ← ← ← ← ← ← ← → → → → → → → → S
Iteration   35 :  S ← ← ← ← ← ← ← ← ← ← → → → → → → → → S
Iteration   40 :  S ← ← ← ← ← ← ← ← ← ← → → → → → → → → S
Iteration   45 :  S ← ← ← ← ← ← ← ← → ← → → → → → → → → S
Iteration   50 :  S ← ← ← ← ← ← ← ← → ← → → → → → → → → S
Iteration   55 :  S ← ← ← ← ← ← ← ← → ← → → → → → → → → S
Iteration   60 :  S ← ← ← ← ← ← ← ← ← → → → → → → → → → S
Iteration   65 :  S ← ← ← ← ← ← ← ← ← → → → → → → → → → S
Iteration   70 :  S ← ← ← ← ← ← ← ← ← → → → → → → → → → S
Iteration   75 :  S ← ← ← ← ← ← ← ← ← → → → → → → → → → S
Iteration   80 :  S ← ← ← ←

The final behaviour is optimal as it will lead the agent to the nearest power point !

### Estimators in large state-action space

The naive table based approach doesn't scale well. That's why we need more sophisticated estimators. One Idea, proposed by Mnih and all in [MniKav2015], is to use a neural network to estimate the $Q$ value of the action based on the visual inputs.

#### Network architecture
The topology preconised in the article is to use a convolutional neural network (CNN) followed by a two layer fully connected feed forward neural net. The CNN is particularily well suited for image analysis, its shared weights helping to detect features independently from localisation in the picture.

#### Training algorithm

The training algorithm for this estimator consists in collecting a labelised dataset from the experiment and using samples from this dataset to adapt the weights of the network. The sampling step is important in order to avoid the high correlation in sequential exemples.

The collection of the dataset require two neural networks. The first one play the role of the state value estimator and the second one is used to choose the best action.

The exploration cycle follow those steps :

1. The agent perceivesthe current state $s$
2. The agent chooses an action $a$ based on an $\epsilon$-greedy policy with respect to the output of the second neural network.
3. The agent receives a reward $r$ and perceives the new state $s'$. 
4. The first neural network is used to estimate the state value $\hat{V}(s')$ of $s'$ (*id est* the maximal $Q$-value achievable from $s'$)
5. The estimated $Q$-value for $(s,a)$ is $r + \gamma\hat{V}(s')$

All the collected tuple $(s, a, r + \gamma\hat{V}(s'))$ are then used as a classical supervised learning case : the weights are ajusted to minimize the error between the second network output for $(s, a)$ and the estimated value $r + \gamma\hat{V}(s')$

Every $k$ steps ($k$ beeing an hyperparameter), the weights of the second network are copied to the first network so that the estimations of $V$ can improve. This is important for the stability of the model to wait for a certain time before updating the state value estimator network. 

<video controls src="videos/Agent.mp4">

### Distributing the exploration phase

In [MniBad2016], Mnih and *al* have proposed a way of getting rid of the replay memory by using multiple agents in parallel to decorrelate the sequences of states.

We are currently working on implementing those solution with python and the theano library but we are facing some problems concerning shared memory, parallel efficiency and python.

## Bibliography

* [MniBad2016] MNIH, Volodymyr, BADIA, Adria Puigdomenech, MIRZA, Mehdi, et al. Asynchronous methods for deep reinforcement learning. arXiv preprint arXiv:1602.01783, 2016.
* [MniKav2015] MNIH, Volodymyr, KAVUKCUOGLU, Koray, SILVER, David, et al. Human-level control through deep reinforcement learning. Nature, 2015, vol. 518, no 7540, p. 529-533.
* [RusNor2010] RUSSELL S., NORVIG P. Intelligence artificielle. 3e édition. Paris: Pearson education France, 2010, 1199p. ISBN 9782744074554
* [StaMii2002] STANLEY, Kenneth O. et MIIKKULAINEN, Risto. Evolving neural networks through augmenting topologies. Evolutionary computation, 2002, vol. 10, no 2, p. 99-127.